In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import FashionMNIST

import random
import copy
import tqdm
import numpy as np
import matplotlib.pyplot as plt

def set_seed(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

In [2]:
class Args():
    device = 'cpu'
    # device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')

    input_size = 784
    output_size = 10
    width = 10
    depth = 5

    batch_size = 128
    lr = 1e-2
    weight_decay = 1e-5
    max_epochs = 100

In [3]:
train_data = FashionMNIST(root='./data', train=True, download=False)
test_data = FashionMNIST(root='./data', train=False, download=False)

X_train = train_data.data.unsqueeze(1).float().flatten(start_dim=1)/255
X_test = test_data.data.unsqueeze(1).float().flatten(start_dim=1)/255

y_train = train_data.targets.long()
y_test = test_data.targets.long()

train_data = torch.utils.data.TensorDataset(X_train, y_train)
test_data = torch.utils.data.TensorDataset(X_test, y_test)

In [4]:
class DNN(nn.Module):

    @staticmethod
    def linear(in_channels, out_channels, bias=False, activation=nn.Identity()):
        return nn.Sequential(
            nn.Linear(in_channels, out_channels, bias=bias),
            activation,
        )

    def __init__(self, input_size, output_size, width, depth):
        super(DNN, self).__init__()

        self.layers = nn.ModuleList()
        for d in range(depth):
            if d == 0:
                self.layers.append(self.linear(input_size, width))
            elif d == depth - 1:
                self.layers.append(self.linear(width, output_size))
            else:
                self.layers.append(self.linear(width, width))

    def init(self, type):
        if not hasattr(self, 'type'):
            self.type = type
        else:
            self.type = 'gaussian'

        for layer in self.layers:
            if self.type == 'gaussian':
                nn.init.normal_(layer[0].weight, mean=0.0, std=1.0)
            else:
                raise NotImplementedError

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [5]:
class Trainer():

    def __init__(self, data, model, optimzer, criterion, args):
        self.args = args
        
        self.traindata, self.testdata = data
        self.trainloader = torch.utils.data.DataLoader(self.traindata, batch_size=self.args.batch_size, shuffle=True, drop_last=True)
        self.testloader = torch.utils.data.DataLoader(self.testdata, batch_size=self.args.batch_size, shuffle=False, drop_last=False)

        self.model = model.to(self.args.device)
        self.optimizer = optimzer
        self.criterion = criterion

        self.loss = []
        self.accuracy = []

    def train_epoch(self):
        train_loss = 0
        train_accuracy = 0

        for idx, (x, y) in tqdm.tqdm(enumerate(self.trainloader)):
            self.model.train()
            x = x.to(self.args.device)
            y = y.to(self.args.device)

            self.optimizer.zero_grad()
            y_pred = self.model(x)
            loss = self.criterion(y_pred, y)
            loss.backward()
            self.optimizer.step()

            train_loss += loss.item()/len(self.trainloader)
            train_accuracy += (y_pred.argmax(dim=1) == y).float().mean().item()/len(self.trainloader)

        return train_loss, train_accuracy
    
    def test(self):
        self.model.eval()
        test_loss = 0
        test_accuracy = 0

        
        for idx, (x, y) in tqdm.tqdm(enumerate(self.testloader)):
            x = x.to(self.args.device)
            y = y.to(self.args.device)

            with torch.no_grad():
                y_pred = self.model(x)
                loss = self.criterion(y_pred, y)

            test_loss += loss.item()/len(self.testloader)
            test_accuracy += (y_pred.argmax(dim=1) == y).float().mean().item()/len(self.testloader)

        return test_loss, test_accuracy
    
    def train(self, epochs):
        for epoch in range(epochs):
            train_loss, train_accuracy = self.train_epoch()
            self.loss.append([train_loss])
            self.accuracy.append([train_accuracy])

            test_loss, test_accuracy = self.test()

            print(f'Epoch: {epoch+1:03d}/{epochs:03d} | Train Loss: {train_loss:.3f} | Train Accuracy: {train_accuracy:.3f} | Test Loss: {test_loss:.3f} | Test Accuracy: {test_accuracy:.3f}')

In [6]:
set_seed(0)
args = Args()

model = DNN(args.input_size, args.output_size, args.width, args.depth)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0)
criterion = nn.CrossEntropyLoss()

trainer = Trainer((train_data, test_data), model, optimizer, criterion, args)
trainer.train(args.max_epochs)

468it [00:00, 968.25it/s]
79it [00:00, 1693.46it/s]


Epoch: 001/100 | Train Loss: 2.169 | Train Accuracy: 0.284 | Test Loss: 1.722 | Test Accuracy: 0.288


468it [00:00, 977.98it/s]
79it [00:00, 1770.74it/s]


Epoch: 002/100 | Train Loss: 1.232 | Train Accuracy: 0.531 | Test Loss: 0.970 | Test Accuracy: 0.604


468it [00:00, 949.15it/s]
79it [00:00, 1719.49it/s]


...

Epoch: 098/100 | Train Loss: 0.419 | Train Accuracy: 0.854 | Test Loss: 0.493 | Test Accuracy: 0.829


468it [00:00, 975.57it/s] 
79it [00:00, 1662.55it/s]


Epoch: 099/100 | Train Loss: 0.420 | Train Accuracy: 0.855 | Test Loss: 0.485 | Test Accuracy: 0.830


468it [00:00, 960.22it/s]
79it [00:00, 1720.16it/s]

Epoch: 100/100 | Train Loss: 0.418 | Train Accuracy: 0.856 | Test Loss: 0.489 | Test Accuracy: 0.828
